# 10.1. Схожесть
Заполните пропуски в следующем коде, для того чтобы получилась система определения близости пользователей и объектов на основе [SVD-разложения](https://colab.research.google.com/drive/1ldt1uUd8bJwiuUHrp2xmTKjELYVLcdzZ#scrollTo=iB0F4K6Kxf2p), как было показано на лекции. 

Для этого требуется реализовать несколько функций, озаглавленных в приведённом ниже шаблоне класса. Функция `_get_svd()` должна выполнять svd-преобразование и возвращать матрицы P и Q (см. [лекцию](https://colab.research.google.com/drive/1aignzmtgTjC8HErXwUJzWlqDcmR8NdCg#scrollTo=LpPUNrmdXf0M)). Функция `get_similar_users` должна ранжировать всех пользователей по мере близости к искомому пользователю и возвращать индексы самых близких из них. Функция `get_similar_items` - аналогично, но по отношению к объектам. В качестве аргументов последние две функции принимают (1) количество похожих пользователей и объектов, индексы которых нужно вернуть, и (2) индекс текущего пользователя или объекта.

In [ ]:

import numpy as np
from numpy.linalg import svd
from sklearn.neighbors import NearestNeighbors


class similaryty_analizer:
    def __init__(self, R: np.array):
        self.R = R
        self.n_users = R.shape[0]
        self.n_items = R.shape[1]

    def _get_svd(self, new_dim: int):
        U, S, V = svd(self.R)

        U = U[:, :new_dim]
        S = np.diag(S[:new_dim])
        V = V[:new_dim, :]

        P = np.matmul(U, S)
        Q = V.T
        return P, Q

    def get_similar_users(self, n_users: int, user_id: int):
        if n_users > self.n_users - 1:
            n_users = self.n_users - 1

        P, Q = self._get_svd(self.n_users)

        nn = NearestNeighbors(n_neighbors=n_users + 1)
        nn = nn.fit(P)

        user = P[user_id, :]
        neighbours = nn.kneighbors(user.reshape(1, -1), return_distance=False)[:, 1:].ravel()
        return neighbours

    def get_similar_items(self, n_items: int, item_id: int):
        if n_items > self.n_items - 1:
            n_items = self.n_items - 1
            
        P, Q = self._get_svd(self.n_items)

        nn = NearestNeighbors(n_neighbors=n_items + 1)
        nn = nn.fit(Q)

        item = Q[item_id, :]
        neighbours = nn.kneighbors(item.reshape(1, -1), return_distance=False)[:, 1:].ravel()
        return neighbours
r1 = [1, 0, 0]
r2 = [1, 1, 0]
r3 = [1, 1, 1]

R = np.array([r1, r2, r3])
R
array([[1, 0, 0],
       [1, 1, 0],
       [1, 1, 1]])
SA = similaryty_analizer(R)
SA.get_similar_users(1, 0)
array([1], dtype=int64)
SA.get_similar_users(2, 0)
array([1, 2], dtype=int64)
SA.get_similar_users(2, 1)
array([0, 2], dtype=int64)

# Пример входных и выходных данных

In [ ]:
r1 = [1, 0, 0]
r2 = [1, 1, 0]
r3 = [1, 1, 1]

R = np.array([r1, r2, r3])
R

array([[1, 0, 0],
       [1, 1, 0],
       [1, 1, 1]])

In [ ]:
SA = similaryty_analizer(R)
SA.get_similar_users(1, 0)

array([1])

In [ ]:
SA.get_similar_users(2, 0)

array([1, 2])

In [ ]:
SA.get_similar_users(2, 1)

array([0, 2])

## Примечания


1. Возвращайте результат в виде `np.array`. В случае использования `list` могут возникнуть ошибки при проверке.

2. В задаче запрещено использовать библиотеку pandas.

3. Обратите внимание, что SVD-преобразование и SVD-разложение - это **разные** операции! В лекциях про это упоминается. SVD-разложение используется для получения трёх матриц: U, S, V. SVD-преобразование используется для снижения размерности. В нём нужно взять от этих матриц первые N строк/столбцов (в данном задании N указан в параметрах функции).